1. Introducción

Cuaderno 00 – Importación y Preparación Inicial de Datos

Este notebook permite cargar de forma flexible cualquier archivo CSV, Excel o base de datos con el histórico de operaciones de una empresa de cambio de divisas.

Su objetivo es:
- Estandarizar el formato de columnas
- Convertir fechas correctamente
- Homogeneizar nombres de divisas
- Preparar la base para el análisis en los siguientes notebooks


2. Carga de librerías

In [12]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración visual (opcional)
plt.style.use("ggplot")
sns.set_palette("Set2")


3. Carga flexible de archivo

    Cargar el archivo entregado:

    Aquí puedes cargar el archivo que te ha entregado la empresa. Aceptamos CSV, Excel o incluso JSON.

In [15]:
# Cargamos el archivo
archivo = "../data/todas_tasas_cambio.csv"
df = pd.read_csv(archivo, encoding="latin1")

# Detección automática de extensión
if archivo.endswith(".csv"):
    df = pd.read_csv(archivo)
elif archivo.endswith(".xlsx"):
    df = pd.read_excel(archivo)
elif archivo.endswith(".json"):
    df = pd.read_json(archivo)
else:
    raise ValueError("Formato de archivo no soportado")

df.head()


,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,GBP=X,2008-08-01,0.50441,0.5068,0.50396,0.50651,0.50651,0.0
1,AUD=X,2008-08-01,1.06430,1.0765,1.06390,1.07270,1.07270,0.0
2,CAD=X,2008-08-01,1.02340,1.0294,1.02290,1.02460,1.02460,0.0
3,CHF=X,2008-08-01,1.04800,1.0512,1.04640,1.05040,1.05040,0.0
4,CNY=X,2008-08-01,6.83600,6.8554,6.82100,6.84700,6.84700,0.0


4. Inspección y estructura básica

    Inspección de columnas y tipos:

    Revisamos nombres, tipos y valores nulos para preparar la normalización.



In [14]:
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38982 entries, 0 to 38981
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Ticker     38982 non-null  object 
 1   Date       38982 non-null  object 
 2   Open       38982 non-null  float64
 3   High       38982 non-null  float64
 4   Low        38982 non-null  float64
 5   Close      38982 non-null  float64
 6   Adj Close  38982 non-null  float64
 7   Volume     38982 non-null  float64
dtypes: float64(6), object(2)
memory usage: 2.4+ MB


Ticker       0
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

5. Normalización de columnas

    Renombrado y estandarización:

    Ejemplo para transformar columnas como:

    - `Fecha de operación` → `fecha`
    - `Moneda origen` → `divisa_origen`
    - `Monto vendido` → `cantidad`


In [ ]:
df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_")

df = df.rename(columns={
    "fecha_operacion": "fecha",
    "divisa_origen": "moneda_origen",
    "divisa_destino": "moneda_destino",
    "monto_vendido": "cantidad",
    "tipo_cambio": "tasa"
})


6. Conversión de fechas

    Conversión de columna de fecha:

    Normalizamos la columna de fecha para asegurar análisis temporal correcto.


In [ ]:
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
df = df.sort_values("fecha").dropna(subset=["fecha"])


7. Exploración rápida del contenido

In [ ]:
# Ver divisas más frecuentes
print("Monedas más frecuentes:", df["moneda_origen"].value_counts().head())

# Ver rango de fechas disponible
print("Desde:", df["fecha"].min(), "| Hasta:", df["fecha"].max())

# Ver tipos de operación si aplica
if "tipo_operacion" in df.columns:
    print("Tipos de operación:", df["tipo_operacion"].unique())


8. Exportación para análisis

    Guardar archivo limpio:

    Guardamos el archivo limpio para usar en los siguientes cuadernos.


In [ ]:
df.to_csv("data/dataset_limpio.csv", index=False)
print("Archivo exportado: data/dataset_limpio.csv")

Este cuaderno te garantiza una entrada robusta sin importar cómo venga el dataset.

9. Importación y filtrado de tasas de cambio históricas

In [ ]:
# Fijamos las divisas objetivo
divisas_objetivo = ["CNY", "JPY", "MAD", "MXN", "TRY", "USD", "EUR"]

# Ruta al archivo que descargaste de Kaggle
ruta_csv_tasas = "data/exchange_rates_usd.csv"
df_tasas_raw = pd.read_csv(ruta_csv_tasas)

# Conversión y filtrado
df_tasas = df_tasas_raw[df_tasas_raw["ISO Code"].isin(divisas_objetivo)].copy()
df_tasas["Date"] = pd.to_datetime(df_tasas["Date"], dayfirst=False)
df_tasas = df_tasas.rename(columns={
    "ISO Code": "divisa",
    "Value (USD)": "tasa_usd",
    "Date": "fecha"
}).sort_values(["divisa", "fecha"]).reset_index(drop=True)

df_tasas.head()

In [ ]:
## Guardamos el resultado limpio para usar más adelante

df_tasas.to_csv("data/tasas_filtradas.csv", index=False)
print("Archivo exportado: data/tasas_filtradas.csv")

10. Convertir tasas USD → EUR

In [ ]:
# Añadimos conversión de tasas desde USD a base EUR
df_tasas["fecha"] = pd.to_datetime(df_tasas["fecha"])

# Extraemos EUR/USD por fecha (tasa inversa: cuánto USD vale 1 EUR)
eur_usd = df_tasas[df_tasas["divisa"] == "EUR"][["fecha", "tasa_usd"]].rename(columns={"tasa_usd": "tasa_eur_usd"})

# Unimos por fecha para tener referencia cruzada
df_tasas = df_tasas.merge(eur_usd, on="fecha", how="inner")

# Calculamos tasa respecto al EUR
df_tasas["tasa_eur"] = df_tasas["tasa_usd"] / df_tasas["tasa_eur_usd"]

# Redondeamos un poco para legibilidad
df_tasas["tasa_eur"] = df_tasas["tasa_eur"].round(5)

# Eliminamos las columnas que ya no necesitamos
df_tasas_final = df_tasas[["fecha", "divisa", "tasa_eur"]].sort_values(["divisa", "fecha"])

df_tasas_final.head()

In [ ]:
# Guardamos la tabla ya normalizada en EUR

df_tasas_final.to_csv("data/tasas_en_eur.csv", index=False)
print("Tasas convertidas a EUR guardadas en: data/tasas_en_eur.csv")